In [1]:
import itertools
import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite, Reaction, Model
import time
import numpy as np
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks

from functools import partial
from src.met_task_functions import get_met_ids, constrain_model, create_reactions


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml'
model = cobra.io.read_sbml_model(model_file_path)

model_list = constrain_model(model, ALLMETSIN=True)

end_time = time.time()
print('Model load and preparation time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpssh3q91t.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpe8415nr0.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpt9yp35xe.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Model load and preparation time: 76.859555 seconds


In [22]:
def tasks_test(task_list: list, model_list: list, gene_ids: list) -> list:
    """Performs knockout FBA and checks tasks for the knockout."""
    with model_list[0]:
        for gene_id in gene_ids:
            try:
                model_list[0].genes.get_by_id(gene_id).knock_out()
            except KeyError:
                return gene_id + ' not in model.'
        res = [model_list[0].slim_optimize()]

    for task in task_list:
        t_model = model_list[task[3]]

        with t_model:
            for subset in [task[0], task[1]]:
                for rx in subset:
                    if rx == 'ALLMETSIN':
                        # Adds boundary metabolites for other reactions when ALLMETSIN is used
                        for r in subset[1:]:
                            for m2 in r.metabolites:
                                for r2 in m2.reactions:
                                    if r2.boundary and r2.id != r.id:
                                        r2.add_metabolites({Metabolite(
                                                            m2.id[:-4] + 'x[x]',
                                                            formula=m2.formula,
                                                            name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                            compartment='x'): 1})
                        continue
                    t_model.add_reaction(rx)

            if task[2] != 'nan':
                t_model.add_reaction(task[2])

            for gene_id in gene_ids:
                t_model.genes.get_by_id(gene_id).knock_out()

            if t_model.slim_optimize(error_value='nan') == 'nan':
                res += [0]
            else:
                res += [1]


    return res

In [4]:
# Read and format task data
tasks_df = pd.read_table('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv')

for b in ['LBin', 'LBout', 'UBin', 'UBout']:
    tasks_df[b] = tasks_df[b].apply(lambda x: x.split(','))

for put in ['inputs', 'outputs']:
    tasks_df[put] = tasks_df[put].apply(lambda x: [e + ']' for e in x[1:-1].split(']')][0:-1])

tasks_df['equations'] = tasks_df['equations'].apply(str)

tasks_df[['met_ids', 'model_num']] = tasks_df.apply(partial(get_met_ids, model_list), axis=1, result_type='expand')

tasks_df = create_reactions(tasks_df)

task_list = list(tasks_df.values.tolist())

In [25]:
# Read test data
test_data = pd.read_csv('C:/Users/Sigve/Genome_Data/results/model_tests/test_data.csv')
test_data['pass/fail'] = test_data.values[:, 4:].tolist()
test_data = test_data[['phewas_code', 'gene_ids', 'solution', 'pass/fail']]
test_data['gene_ids'] = test_data['gene_ids'].apply(lambda x: x.split(','))
print(test_data.head())

# Reduce number of entries
#test_data = test_data.iloc[:10, :]

   phewas_code                            gene_ids   solution  \
0       290.11  [ENSG00000100030, ENSG00000197375] -80.729783   
1       290.11  [ENSG00000101577, ENSG00000197375] -80.729783   
2       290.11  [ENSG00000102780, ENSG00000197375] -80.729783   
3       290.11  [ENSG00000107798, ENSG00000197375] -80.729783   
4       290.11  [ENSG00000113448, ENSG00000197375] -80.729783   

                                           pass/fail  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  


In [18]:
g = ['ENSG00000100030', 'ENSG00000197375']
%load_ext line_profiler
%lprun -f tasks_test tasks_test(task_list, model_list, g)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
*** KeyboardInterrupt exception caught in code being profiled.

In [26]:
start_time = time.time()

test_data['results'] = test_data['gene_ids'].apply(partial(tasks_test, task_list, model_list))

end_time = time.time()
print('FBA runtime: %.6f seconds' % (end_time - start_time))

FBA runtime: 3719.969618 seconds


In [27]:
# Data comparison

result_df = test_data.copy()

print(result_df.shape[0])


result_df['main_obj'] = result_df['results'].apply(lambda x: x[0])
result_df['results'] = result_df['results'].apply(lambda x: x[1:])

result_df['comparison'] = result_df[['pass/fail', 'results']].apply(lambda x: any([False if i == j else True for i, j in zip(x[0], x[1])]), axis=1)
error_df = result_df[result_df['comparison']]
print('Number of detected errors: ' + str(error_df.shape[0]))


204
Number of detected errors: 0
